<a href="https://colab.research.google.com/github/3hoodaas/Mobadron_AI/blob/main/Mobadron_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# الخطوة 1: استيراد المكتبات
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report
import joblib

# الخطوة 2: تحميل البيانات

file_path = '/content/Fake_Initiatives_Dataset.xlsx'  # استبدل هذا بالمسار الخاص بك
data = pd.read_excel(file_path)

# عرض أول 5 صفوف من البيانات
print("عرض أول 5 صفوف من البيانات:")
print(data.head())

# الخطوة 3: تنظيف البيانات
# التأكد من عدم وجود قيم مفقودة
print("\nالتأكد من القيم المفقودة:")
print(data.isnull().sum())

# إذا كانت هناك قيم مفقودة، يمكن استبدالها أو حذفها
# حذف الصفوف التي تحتوي على قيم مفقودة
data.dropna(inplace=True)
print("\nبعد تنظيف القيم المفقودة:")
print(data.isnull().sum())

# الخطوة 4: معالجة النصوص باستخدام LabelEncoder
label_encoder = LabelEncoder()

# الأعمدة النصية التي تحتاج إلى تحويل
text_columns = ['Target_Group', 'Expected_Impact']  # مثال
for col in text_columns:
    data[col] = label_encoder.fit_transform(data[col])

# الخطوة 5: إضافة عمود للأولوية باستخدام النقاط الموزونة بدون عمود Time
def calculate_priority_weighted_no_time(row):
    # تحديد نقاط الميزانية
    if row['Budget'] > 3000000:
        budget_score = 10
    elif row['Budget'] > 2000000:
        budget_score = 7
    elif row['Budget'] > 1000000:
        budget_score = 5
    else:
        budget_score = 3

    # تحديد نقاط عدد المستفيدين
    if row['Beneficiaries'] > 20000:
        beneficiaries_score = 10
    elif row['Beneficiaries'] > 10000:
        beneficiaries_score = 7
    elif row['Beneficiaries'] > 5000:
        beneficiaries_score = 5
    else:
        beneficiaries_score = 3

    # تحديد نقاط التأثير المتوقع
    if row['Expected_Impact'] > 50000:
        impact_score = 10
    elif row['Expected_Impact'] > 20000:
        impact_score = 7
    elif row['Expected_Impact'] > 10000:
        impact_score = 5
    else:
        impact_score = 3

    # حساب النقاط النهائية باستخدام الأوزان
    final_score = (0.333 * budget_score) + (0.333 * beneficiaries_score) + \
                  (0.333 * impact_score)

    # تحديد الأولوية بناءً على النقاط النهائية
    if final_score > 8:
        return 1  # أولوية عالية
    elif final_score >= 5:
        return 2  # أولوية متوسطة
    else:
        return 3  # أولوية منخفضة

# تطبيق المعادلة الجديدة على البيانات
data['Priority'] = data.apply(calculate_priority_weighted_no_time, axis=1)


# الخطوة 6: إعداد الميزات والهدف
features = ['Target_Group', 'Expected_Impact', 'Budget', 'Beneficiaries']  # الميزات الأساسية
X = data[features]
y = data['Priority']

# تقسيم البيانات إلى بيانات تدريب واختبار
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# تجهيز الميزات باستخدام StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# الخطوة 7: إنشاء النموذج
model = RandomForestClassifier(random_state=42)

# تدريب النموذج
model.fit(X_train_scaled, y_train)

# التنبؤ بالاختبار
y_pred = model.predict(X_test_scaled)

# الخطوة 8: تقييم الأداء
print("\nتقرير الأداء للنموذج:")
print(classification_report(y_test, y_pred))

# الخطوة 9: حفظ النموذج
model_path = '/content/recommendation_model.pkl'
joblib.dump(model, model_path)
print(f"\nتم حفظ النموذج في: {model_path}")

# الخطوة 10: حفظ البيانات المعدلة
output_file_path = '/content/Modified_Initiatives_with_Priority.xlsx'
data.to_excel(output_file_path, index=False)
print(f"\nتم حفظ البيانات المعدلة في: {output_file_path}")


عرض أول 5 صفوف من البيانات:
  Initiative_Name Target_Group Expected_Impact     Budget  Beneficiaries
0    Initiative 1     Startups           78434  3089471.0        14013.0
1    Initiative 2        Women           67784  2055760.0        20398.0
2    Initiative 3     Startups           74902  4267869.0         7937.0
3    Initiative 4        Women           29707  2056631.0        45692.0
4    Initiative 5        Women           18907   354684.0        13875.0

التأكد من القيم المفقودة:
Initiative_Name     0
Target_Group        0
Expected_Impact    26
Budget             40
Beneficiaries      26
dtype: int64

بعد تنظيف القيم المفقودة:
Initiative_Name    0
Target_Group       0
Expected_Impact    0
Budget             0
Beneficiaries      0
dtype: int64

تقرير الأداء للنموذج:
              precision    recall  f1-score   support

           2       0.99      0.99      0.99        81
           3       0.95      0.95      0.95        20

    accuracy                           0.98       10

In [8]:
# الخطوة 1: استيراد المكتبات
import pandas as pd

# متغير لتسجيل عدد التكرارات والجديدات
duplicate_count = 0
new_count = 0

# الخطوة 2: تحميل البيانات المرجعية
reference_file_path = '/content/Fake_Initiatives_Dataset.xlsx'
try:
    reference_data = pd.read_excel(reference_file_path)
    print("\nتم تحميل البيانات المرجعية بنجاح.")
except FileNotFoundError:
    print("\nلا يوجد ملف مرجعي، سيتم إنشاؤه عند إضافة أول مبادرة جديدة.")
    reference_data = pd.DataFrame(columns=['Initiative_Name', 'Target_Group', 'Budget', 'Beneficiaries', 'Expected_Impact'])

# الخطوة 3: وظيفة لفحص التكرار
def is_duplicate(new_initiative, reference_data):
    """
    تحقق مما إذا كانت المبادرة الجديدة مكررة بناءً على البيانات المرجعية.
    """
    duplicate = reference_data[
        (reference_data['Initiative_Name'] == new_initiative['Initiative_Name']) &
        (reference_data['Target_Group'] == new_initiative['Target_Group']) &
        (reference_data['Budget'] == new_initiative['Budget']) &
        (reference_data['Beneficiaries'] == new_initiative['Beneficiaries']) &
        (reference_data['Expected_Impact'] == new_initiative['Expected_Impact'])
    ]
    return not duplicate.empty  # إرجاع True إذا كانت مكررة

# الخطوة 4: إضافة مبادرة جديدة مع حساب التكرارات والجديدات
def add_new_initiative_with_metrics(new_initiative, reference_data, reference_file_path):
    global duplicate_count, new_count
    if is_duplicate(new_initiative, reference_data):
        duplicate_count += 1
        print("\nالمبادرة مكررة، لا يمكن تسجيلها.")
        return reference_data
    else:
        new_count += 1
        print("\nالمبادرة جديدة، سيتم إضافتها.")
        new_initiative_df = pd.DataFrame([new_initiative])
        reference_data = pd.concat([reference_data, new_initiative_df], ignore_index=True)
        reference_data.to_excel(reference_file_path, index=False)
        print("\nتم تحديث البيانات المرجعية.")
        return reference_data

# الخطوة 5: مثال لإضافة مبادرات جديدة
new_initiatives = [
    {'Initiative_Name': 'New Initiative 1', 'Target_Group': 'Startups', 'Budget': 2000000, 'Beneficiaries': 10000, 'Expected_Impact': 50000},
    {'Initiative_Name': 'New Initiative 2', 'Target_Group': 'Women', 'Budget': 1500000, 'Beneficiaries': 8000, 'Expected_Impact': 30000},
    {'Initiative_Name': 'New Initiative 1', 'Target_Group': 'Startups', 'Budget': 2000000, 'Beneficiaries': 10000, 'Expected_Impact': 50000}  # مكررة
]

# إعادة تعيين القيم قبل التشغيل
duplicate_count = 0
new_count = 0

# تشغيل الكود مع تسجيل النتائج
for initiative in new_initiatives:
    reference_data = add_new_initiative_with_metrics(initiative, reference_data, reference_file_path)

# حساب نسبة الأداء
total_initiatives = duplicate_count + new_count
if total_initiatives > 0:
    duplicate_percentage = (duplicate_count / total_initiatives) * 100
    new_percentage = (new_count / total_initiatives) * 100
    print(f"\nعدد المبادرات: {total_initiatives}")
    print(f"النسبة المئوية للمبادرات المكررة: {duplicate_percentage:.2f}%")
    print(f"النسبة المئوية للمبادرات الجديدة: {new_percentage:.2f}%")
else:
    print("\nلا توجد مبادرات لتحليل الأداء.")



تم تحميل البيانات المرجعية بنجاح.

المبادرة جديدة، سيتم إضافتها.

تم تحديث البيانات المرجعية.

المبادرة جديدة، سيتم إضافتها.

تم تحديث البيانات المرجعية.

المبادرة مكررة، لا يمكن تسجيلها.

عدد المبادرات: 3
النسبة المئوية للمبادرات المكررة: 33.33%
النسبة المئوية للمبادرات الجديدة: 66.67%


In [11]:
# الخطوة 1: استيراد المكتبات
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

# الخطوة 2: تحميل البيانات
file_path = '/content/Fake_Initiatives_Dataset.xlsx'
try:
    data = pd.read_excel(file_path)
    print("\nتم تحميل البيانات بنجاح.")
except FileNotFoundError:
    print("\nالملف غير موجود. يرجى توفير ملف البيانات.")
    data = pd.DataFrame(columns=['Initiative_Name', 'Target_Group', 'Budget', 'Beneficiaries', 'Expected_Impact'])

# الخطوة 3: ترميز النصوص باستخدام LabelEncoder
label_encoder = LabelEncoder()
text_columns = ['Initiative_Name', 'Target_Group']
for col in text_columns:
    data[col] = label_encoder.fit_transform(data[col])

# الخطوة 4: إنشاء عمود 'Is_Duplicate'
def mark_duplicates(row, reference_data):
    duplicate = reference_data[
        (reference_data['Initiative_Name'] == row['Initiative_Name']) &
        (reference_data['Target_Group'] == row['Target_Group']) &
        (reference_data['Budget'] == row['Budget']) &
        (reference_data['Beneficiaries'] == row['Beneficiaries']) &
        (reference_data['Expected_Impact'] == row['Expected_Impact'])
    ]
    return 1 if not duplicate.empty else 0

# استخدام البيانات لإنشاء عمود "Is_Duplicate"
data['Is_Duplicate'] = data.apply(lambda row: mark_duplicates(row, data), axis=1)

# الخطوة 5: إعداد الميزات والهدف
features = ['Initiative_Name', 'Target_Group', 'Budget', 'Beneficiaries', 'Expected_Impact']
X = data[features]
y = data['Is_Duplicate']

# تقسيم البيانات إلى تدريب واختبار
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# الخطوة 6: تدريب نموذج Random Forest
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# الخطوة 7: التنبؤ وتقييم الأداء
y_pred = model.predict(X_test)
print("\nتقرير الأداء:")
print(classification_report(y_test, y_pred))

# الخطوة 8: حفظ النموذج (اختياري)
import joblib
model_path = '/content/duplicate_detection_model.pkl'
joblib.dump(model, model_path)
print(f"\nتم حفظ النموذج في: {model_path}")



تم تحميل البيانات بنجاح.


ValueError: could not convert string to float: ' '

In [ ]:
# الخطوة 1: استيراد المكتبات
import firebase_admin
from firebase_admin import credentials, firestore
import pandas as pd

# الخطوة 2: تهيئة Firebase
# تحميل مفتاح الخدمة من Firebase
cred = credentials.Certificate('/content/mobadron-dc88e-firebase-adminsdk-waxeo-ca9bba9170.json')
firebase_admin.initialize_app(cred)

# الاتصال بـ Firestore
db = firestore.client()

# الخطوة 3: وظيفة لفحص التكرار
def is_duplicate(new_initiative):
    """
    تحقق مما إذا كانت المبادرة الجديدة مكررة بناءً على البيانات المرجعية في Firebase.
    """
    initiatives_ref = db.collection('initiatives')
    query = initiatives_ref.where('Initiative_Name', '==', new_initiative['Initiative_Name']) \
                           .where('Target_Group', '==', new_initiative['Target_Group']) \
                           .where('Budget', '==', new_initiative['Budget']) \
                           .where('Beneficiaries', '==', new_initiative['Beneficiaries']) \
                           .where('Expected_Impact', '==', new_initiative['Expected_Impact'])
    results = query.get()
    return len(results) > 0  # إرجاع True إذا كانت مكررة

# الخطوة 4: إضافة مبادرة جديدة
def add_new_initiative(new_initiative):
    """
    إضافة مبادرة جديدة إلى Firebase بعد التحقق من التكرار.
    """
    if is_duplicate(new_initiative):
        print("\nالمبادرة مكررة، لا يمكن تسجيلها.")
    else:
        print("\nالمبادرة جديدة، سيتم إضافتها.")
        # إضافة المبادرة إلى Firebase
        db.collection('initiatives').add(new_initiative)
        print("\nتم تسجيل المبادرة بنجاح.")

# الخطوة 5: مثال لإضافة مبادرات جديدة
new_initiatives = [
    {'Initiative_Name': 'New Initiative 1', 'Target_Group': 'Startups', 'Budget': 2000000, 'Beneficiaries': 10000, 'Expected_Impact': 50000},
    {'Initiative_Name': 'New Initiative 2', 'Target_Group': 'Women', 'Budget': 1500000, 'Beneficiaries': 8000, 'Expected_Impact': 30000},
    {'Initiative_Name': 'New Initiative 1', 'Target_Group': 'Startups', 'Budget': 2000000, 'Beneficiaries': 10000, 'Expected_Impact': 50000}  # مكررة
]

# إضافة كل مبادرة جديدة مع التحقق من التكرار
for initiative in new_initiatives:
    add_new_initiative(initiative)


/usr/local/lib/python3.10/dist-packages/google/cloud/firestore_v1/base_collection.py:304: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)
<ipython-input-16-43c6fe265599>:21: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  .where('Target_Group', '==', new_initiative['Target_Group']) \
<ipython-input-16-43c6fe265599>:22: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  .where('Budget', '==', new_initiative['Budget']) \
<ipython-input-16-43c6fe265599>:23: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  .where('Beneficiaries', '==', new_initiative['Beneficiaries']) \
<ipython-input-16-43c6fe265599>:24: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword arg


المبادرة جديدة، سيتم إضافتها.

تم تسجيل المبادرة بنجاح.

المبادرة جديدة، سيتم إضافتها.

تم تسجيل المبادرة بنجاح.

المبادرة مكررة، لا يمكن تسجيلها.


In [ ]:
# الخطوة 1: استيراد المكتبات
import firebase_admin
from firebase_admin import credentials, firestore

# الخطوة 2: تهيئة Firebase
# تحميل مفتاح الخدمة من Firebase
cred = credentials.Certificate('/content/mobadron-dc88e-firebase-adminsdk-waxeo-ca9bba9170.json')
firebase_admin.initialize_app(cred)

# الاتصال بـ Firestore
db = firestore.client()

# الخطوة 3: وظيفة لفحص التكرار
def is_duplicate(new_initiative):
    """
    تحقق مما إذا كانت المبادرة الجديدة مكررة بناءً على البيانات المرجعية في Firebase.
    """
    initiatives_ref = db.collection('initiatives')
    query = (
        initiatives_ref.filter('Initiative_Name', '==', new_initiative['Initiative_Name'])
        .filter('Target_Group', '==', new_initiative['Target_Group'])
        .filter('Budget', '==', new_initiative['Budget'])
        .filter('Beneficiaries', '==', new_initiative['Beneficiaries'])
        .filter('Expected_Impact', '==', new_initiative['Expected_Impact'])
    )
    results = query.get()
    return len(results) > 0  # إرجاع True إذا كانت مكررة

# الخطوة 4: إضافة مبادرة جديدة
def add_new_initiative(new_initiative):
    """
    إضافة مبادرة جديدة إلى Firebase بعد التحقق من التكرار.
    """
    if is_duplicate(new_initiative):
        print("\nالمبادرة مكررة، لا يمكن تسجيلها.")
    else:
        print("\nالمبادرة جديدة، سيتم إضافتها.")
        # إضافة المبادرة إلى Firebase
        db.collection('initiatives').add(new_initiative)
        print("\nتم تسجيل المبادرة بنجاح.")

# الخطوة 5: مثال لإضافة مبادرات جديدة
new_initiatives = [
    {'Initiative_Name': 'New Initiative 1', 'Target_Group': 'Startups', 'Budget': 2000000, 'Beneficiaries': 10000, 'Expected_Impact': 50000},
    {'Initiative_Name': 'New Initiative 2', 'Target_Group': 'Women', 'Budget': 1500000, 'Beneficiaries': 8000, 'Expected_Impact': 30000},
    {'Initiative_Name': 'New Initiative 1', 'Target_Group': 'Startups', 'Budget': 2000000, 'Beneficiaries': 10000, 'Expected_Impact': 50000}  # مكررة
]

# إضافة كل مبادرة جديدة مع التحقق من التكرار
for initiative in new_initiatives:
    add_new_initiative(initiative)


ValueError: The default Firebase app already exists. This means you called initialize_app() more than once without providing an app name as the second argument. In most cases you only need to call initialize_app() once. But if you do want to initialize multiple apps, pass a second argument to initialize_app() to give each app a unique name.